# Check Fox Glacier Landslide data when merging manual retrieval and automatic retrieval

Manual retrieval is using cr6-downloader or crx, **not** retrieval when at the site.

For the manual data there were no sensor changes.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
%matplotlib notebook

In [ ]:
logger = 'foxglacierlandslide'

## Reading and preparing data

### Split data file into like parts.

Some sites will have changes to instruments and this may change numbers of columns as well as column content

This will cause issues reading the file into a dataframe, as further header lines and number of columns will cause an error.

In [ ]:
#remove any remaining split files
try:
    files = glob.glob('xx*')
    for file in files:
        os.remove(file)
except:
    pass
              
loggerfile = logger+'_Table1.csv'
    
#split file so one part for each header line
cmd = 'csplit -z '+loggerfile+' /RecNbr/ {*}'
os.system(cmd)

files = glob.glob('xx*')
files.sort()
loggerfile = files[-1]
print (logger+' datafile contains '+str(len(files))+' components, displaying the only the most recent')

### Read data

In [ ]:
data = pd.read_csv(loggerfile, parse_dates=True, index_col = 'Time', na_values='NAN')
data.head()

In [ ]:
olddata1 = pd.read_csv('/home/sherburn/datalogger_manualretrieval_archive/foxglacierlandslide_20190206-20190516.csv', parse_dates=True, index_col = 'Time', na_values='NAN')
olddata1.head()

### Plot supply voltage

In [ ]:
vol = data['SupplyVolts'].plot(figsize=(20,7), title='Supply Voltage - '+logger, fontsize=12)
olddata1['SupplyVolts'].plot(ax=vol, color='red')
vol.title.set_size(20)
vol.grid()
vol.set_ylim(11,15)

### Plot observation data

In [ ]:
fig,(ax0, ax1, ax2,ax3) = plt.subplots(4, 1, figsize=(20,20))
plt.subplots_adjust(wspace=0.5, hspace=0.5)

data['PTemp_degC'].plot(ax=ax0, title='Box Temperature - '+logger, fontsize=12)
olddata1['PTemp_degC'].plot(ax=ax0, color='red')
ax0.title.set_size(20)
ax0.grid()
ax0.set_ylabel('Temperature (deg C)')
ax0.set_xlabel('')

ax1.vlines(data.index, ymax=data['Rain1_mm_Tot'], ymin=0, color='blue')
ax1.vlines(olddata1.index, ymax=olddata1['Rain1_mm_Tot'], ymin=0, color='red')
ax1.set_ylabel('Rainfall (mm)')
ax1.set_title('Rainfall Sensor 1')
ax1.title.set_size(20)
ax1.grid()

ax2.vlines(data.index, ymax=data['Rain2_mm_Tot'], ymin=0, color='blue')
ax2.vlines(olddata1.index, ymax=olddata1['Rain2_mm_Tot'], ymin=0, color='red')
ax2.set_ylabel('Rainfall (mm)')
ax2.set_title('Rainfall Sensor 2')
ax2.title.set_size(20)
ax2.grid()

#daily rainfall totals
daily = data['Rain1_mm_Tot'].resample('D').sum()
ax3.vlines(daily.index, ymax=daily, ymin=0, color='blue')
olddaily1 = olddata1['Rain1_mm_Tot'].resample('D').sum()
ax3.vlines(olddaily1.index, ymax=olddaily1, ymin=0, color='red')
ax3.set_ylabel('Rainfall (mm)')
ax3.set_title('Rainfall Sensor 1, Daily Total (UTC Day)')
ax3.title.set_size(20)
ax3.grid()